In [ ]:
import json

file_name = '/Users/ay227/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com/My Drive/Final Year Experiments/Thesis-Experiments/Experiments/BaseLineExperiment/ANERCorp_CamelLab_arabertv02/fine_tuning/evaluation_metrics.json'
corpus_file = '/Users/ay227/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com/My Drive/Final Year Experiments/Thesis-Experiments/Experiments/ExperimentData/corpora.json'
with open(file_name, 'r') as file:
    entity_outputs = json.load(file)  # Use json.load() to read file, not json.loads()
    
with open(corpus_file, 'r') as file:
    corpora = json.load(file)  # Use json.load() to read file, not json.loads()


In [ ]:
import logging
from abc import ABC, abstractmethod
from dataclasses import dataclass, field

import numpy as np
import pandas as pd
from seqeval.metrics import classification_report as seq_classification
from seqeval.metrics import f1_score as seq_f1
from seqeval.metrics import precision_score as seq_precision
from seqeval.metrics import recall_score as seq_recall
from sklearn.metrics import classification_report as skl_classification
from sklearn.metrics import f1_score as skl_f1
from sklearn.metrics import precision_score as skl_precision
from sklearn.metrics import recall_score as skl_recall
from torch import nn
from seqeval.scheme import IOB1, IOB2, IOE1, IOE2, IOBES, BILOU, auto_detect

VALID_SCHEMES = {
    'IOB1':IOB1,
    'IOB2':IOB2,
    'IOE1':IOE1,
    'IOE2':IOE2,
    'IOBES':IOBES,
    'BILOU':BILOU
    }
class EvaluationStrategy(ABC):
    def __init__(self, inv_map):
        self.inv_map = inv_map
        self.ignore_index = nn.CrossEntropyLoss().ignore_index

    def align_predictions(self, predictions, truth):
        predictions = np.argmax(predictions, axis=2)
        batch_size, seq_len = predictions.shape

        truth_list = [[] for _ in range(batch_size)]
        pred_list = [[] for _ in range(batch_size)]

        for i in range(batch_size):
            for j in range(seq_len):
                if truth[i, j] != self.ignore_index:
                    truth_list[i].append(self.inv_map[truth[i][j]])
                    pred_list[i].append(self.inv_map[predictions[i][j]])
                    
        if len(truth_list) != len(pred_list):
            raise ValueError("Aligned predictions and truth have mismatched lengths.")
        return truth_list, pred_list

    def create_classification_report(self, results):
        lines = []
        for line in results.strip().split("\n")[1:]:
            if line.strip():
                tokens = line.split()
                # Remove intermediate aggregation if exists (multi-class)
                if len(tokens) > 5:
                    del tokens[1]
                lines.append(tokens)
        report = pd.DataFrame(
            lines, columns=["Tag", "Precision", "Recall", "F1", "Support"]
        )
        return report

class TokenEvaluationStrategy(EvaluationStrategy):
    def compute_metrics(self, true_labels, predictions):
        try:
            truth_list, pred_list = self.align_predictions(predictions, true_labels)
        except:
            logging.info('The labels already aligned, proceed with evaluation')
            truth_list, pred_list = true_labels, predictions
        
        flat_truth = [item for sublist in truth_list for item in sublist]
        flat_preds = [item for sublist in pred_list for item in sublist]
        report = skl_classification(y_true=flat_truth, y_pred=flat_preds, digits=4)
        report = self.create_classification_report(report)
        cleaned_report = self.clean_report(report)
        return {
            "Precision": skl_precision(
                y_true=flat_truth, y_pred=flat_preds, average="macro"
            ),
            "Recall": skl_recall(
                y_true=flat_truth, y_pred=flat_preds, average="macro"
                ),
            "F1": skl_f1(
                y_true=flat_truth, y_pred=flat_preds, average="macro"
                ),
            "classification": cleaned_report,
            "output": {"y_true": flat_truth, "y_pred": flat_preds},
        }

    def clean_report(self, report):
        report = report.copy()
        mask = report["Tag"] == "accuracy"
        accuracy_row = report[mask]
        if not accuracy_row.empty:
            # Get the accuracy value
            accuracy_value = accuracy_row["Precision"].values[
                0
            ]  # Assuming accuracy is stored in the 'Precision' column
            accuracy_support = accuracy_row["Recall"].values[
                0
            ]  # Assuming accuracy is stored in the 'Precision' column

            # Set the precision, recall, and F1-score to the accuracy value
            report.loc[mask, "Precision"] = accuracy_value
            report.loc[mask, "Recall"] = accuracy_value
            report.loc[mask, "F1"] = accuracy_value
            report.loc[mask, "Support"] = accuracy_support

            # Rename the tag from 'accuracy' to 'accuracy/micro' for clarity
            report.loc[report["Tag"] == "accuracy", "Tag"] = "accuracy/micro"
        return report
    
class EntityEvaluationStrategy(EvaluationStrategy):
    def compute_metrics(self, true_labels, predictions, entity_config):
        scheme = entity_config.get('scheme')  # Default to 'none' if not specified

        # Check if the scheme is valid and not 'none'
        try:
            truth_list, pred_list = self.align_predictions(predictions, true_labels)
        except:
            logging.info('The labels already aligned, proceed with evaluation')
            truth_list, pred_list = true_labels, predictions
            
        strict_outputs = self._evaluate_strict(truth_list, pred_list, scheme)
        non_strict_outputs = self._evaluate_non_strict(truth_list, pred_list)
        
        return {
            "strict": strict_outputs,
            "non_strict": non_strict_outputs,
            "output": {"y_true": truth_list, "y_pred": pred_list}
        }
        
        
    
    def _evaluate_strict(self, truth_list, pred_list, scheme):
        
        if scheme is not None and scheme in VALID_SCHEMES:
            scheme_class = VALID_SCHEMES[scheme]
            report = seq_classification(
                    y_true=truth_list,
                    y_pred=pred_list,
                    digits=4,
                    mode='strict',
                    scheme=scheme_class,
                )
            precision = seq_precision(
                    y_true=truth_list, y_pred=pred_list, average="micro", mode='strict', scheme = scheme_class
                )
            recall = seq_recall(
                    y_true=truth_list, y_pred=pred_list, average="micro", mode='strict', scheme = scheme_class
                )
            f1 = seq_f1(
                    y_true=truth_list, y_pred=pred_list, average="micro", mode='strict', scheme = scheme_class
                )
            
        else:
            logging.info("The scheme is unspecified; seqeval will auto-detect the scheme.")
            report = seq_classification(
                    y_true=truth_list,
                    y_pred=pred_list,
                    digits=4,
                    mode='strict',
                )
            scheme_class = auto_detect(pred_list, False)
            precision = seq_precision(
                    y_true=truth_list, y_pred=pred_list, average="micro", mode='strict', scheme = scheme_class
                )
            recall = seq_recall(
                    y_true=truth_list, y_pred=pred_list, average="micro", mode='strict', scheme = scheme_class
                )
            f1 = seq_f1(
                    y_true=truth_list, y_pred=pred_list, average="micro", mode='strict', scheme = scheme_class
                )
        
        return {
            "Precision": precision,
            "Recall": recall,
            "F1": f1,
            "classification": self.create_classification_report(report),
                
            }
    
        
    def _evaluate_non_strict(self, truth_list, pred_list,):
        
        report = seq_classification(
                    y_true=truth_list,
                    y_pred=pred_list,
                    digits=4,
                )
        precision = seq_precision(
                y_true=truth_list, y_pred=pred_list, average="micro"
            )
        recall = seq_recall(
                    y_true=truth_list, y_pred=pred_list, average="micro"
                )
        f1 = seq_f1(
                y_true=truth_list, y_pred=pred_list, average="micro"
            )
        return {
            "Precision": precision,
            "Recall": recall,
            "F1": f1,
            "classification": self.create_classification_report(report),
                
            }



class Evaluation:
    def __init__(self, inv_map, y_true, y_pred, evaluation_config):
        self.truths = y_true
        self.predictions = y_pred
        self.evaluation_config = evaluation_config
        self.token_strategy = TokenEvaluationStrategy(inv_map)
        self.entity_strategy = EntityEvaluationStrategy(inv_map)

    

    def evaluate(self):
        token_metrics = self.token_strategy.compute_metrics(
            self.truths, self.predictions
        )
        entity_metrics = self.entity_strategy.compute_metrics(
            self.truths, self.predictions, self.evaluation_config
        )

        # Combine or store results as needed
        return {"Token_Level": token_metrics, "Entity_Level": entity_metrics}

    def _prepare_results(self, metrics):
        results = pd.DataFrame.from_dict(self._round_and_slice(metrics))
        report = metrics["classification"]
        output = metrics["output"]
        return results, report, output
    
    def _prepare_entity_results(self, metrics):
        strict = metrics["strict"]
        non_strict = metrics["non_strict"]
        entity_strict_results = pd.DataFrame.from_dict(self._round_and_slice(strict))
        entity_non_strict_results = pd.DataFrame.from_dict(self._round_and_slice(non_strict))
        entity_strict_report = strict['classification']
        entity_non_strict_report = non_strict['classification']
        output = metrics["output"]
        return {
            'entity_strict_results': entity_strict_results,
            'entity_non_strict_results': entity_non_strict_results,
            'entity_strict_report': entity_strict_report,
            'entity_non_strict_report': entity_non_strict_report,
            'output': output,
        }

    def _round_and_slice(self, dictionary):
        # Slicing and rounding results for cleaner presentation
        
        keys_for_slicing = ["Precision", "Recall", "F1"]
        sliced_dict = {key: [round(dictionary[key], 4)] for key in keys_for_slicing}
        return sliced_dict
    
    def generate_results(self):
        metrics = self.evaluate()
        token_results, token_report, token_outputs = self._prepare_results(
            metrics["Token_Level"]
        )
        entity_level_outputs = self._prepare_entity_results(
            metrics["Entity_Level"]
        )
        
        

        return {
            "token_results": token_results,
            "token_report": token_report,
            "token_outputs": token_outputs,
            "entity_strict_results": entity_level_outputs['entity_strict_results'],
            "entity_non_strict_results": entity_level_outputs['entity_non_strict_results'],
            "entity_strict_report": entity_level_outputs['entity_strict_report'],
            "entity_non_strict_report": entity_level_outputs['entity_non_strict_report'],
            "entity_outputs": entity_level_outputs['output'],
        }


@dataclass
class Metrics:
    token_results: pd.DataFrame = field(default_factory=pd.DataFrame)
    token_report: pd.DataFrame = field(default_factory=pd.DataFrame)
    token_outputs: dict = field(default_factory=dict)
    entity_strict_results: pd.DataFrame = field(default_factory=pd.DataFrame)
    entity_non_strict_results: pd.DataFrame = field(default_factory=pd.DataFrame)
    entity_strict_report: pd.DataFrame = field(default_factory=pd.DataFrame)
    entity_non_strict_report: pd.DataFrame = field(default_factory=pd.DataFrame)
    entity_outputs: dict = field(default_factory=dict)

    @staticmethod
    def from_dict(data: dict):
        """Create an instance from a dictionary."""
        required_keys = [
        "token_results", "token_report", "token_outputs",
        "entity_strict_results", "entity_non_strict_results",
        "entity_strict_report", "entity_non_strict_report", "entity_outputs"
        ]
        missing_keys = [key for key in required_keys if key not in data]
        if missing_keys:
            raise ValueError(f"Missing required keys in data: {missing_keys}")
        return Metrics(**data)

    def to_dict(self):
        return {
            "token_results": self.token_results.to_dict(orient="records"),
            "token_report": self.token_report.to_dict(orient="records"),
            "token_outputs": self.token_outputs,
            "entity_strict_results": self.entity_strict_results.to_dict(orient="records"),     
            "entity_non_strict_results": self.entity_non_strict_results.to_dict(orient="records"),
            "entity_strict_report": self.entity_strict_report.to_dict(orient="records"), 
            "entity_non_strict_report": self.entity_non_strict_report.to_dict(orient="records"), 
            "entity_outputs": self.entity_outputs,
        }


In [ ]:
y_true = entity_outputs['entity_outputs']['y_true']
y_pred = entity_outputs['entity_outputs']['y_pred']
corpus = corpora['ANERCorp_CamelLab']
labels_map = corpus["labels_map"]
inv_labels_map = {v: k for k, v in labels_map.items()}
evaluation_config = {
	'scheme': 'IOB2'
}


In [ ]:
evaluator = Evaluation(
                inv_labels_map, y_true, y_pred, evaluation_config
            )
results = evaluator.generate_results()
metrics = Metrics.from_dict(results)

In [ ]:
metrics.entity_strict_results

In [ ]:
outs = evaluator.evaluate()


In [ ]:

outs['Entity_Level']['strict']

In [ ]:
from seqeval.metrics import f1_score, classification_report, f1_score
from seqeval.scheme import IOB2

print(classification_report(y_true, y_pred, mode='strict', digits=4))

f1_score(y_true, y_pred, mode='strict', scheme=IOB2,  average='micro')


In [ ]:
from seqeval.metrics import f1_score, classification_report

print(classification_report(y_true, y_pred, mode=None, digits=4))
